### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
%autosave 60

Autosaving every 60 seconds


In [2]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

## Player Count

* Display the total number of players


In [3]:
unique_players = purchase_data["SN"].unique()
unique_player_count = len(unique_players)
pd.DataFrame([
    {"Total Players": len(unique_players)}
])

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [4]:
moneyFormat = "${:,.2f}".format

def basicDFStyle(df, money_columns=[], bold_columns=[]):
    return df.style.format("${:,.2f}", pd.IndexSlice[:, money_columns])\
        .applymap(lambda x:"font-weight: bold", subset=pd.IndexSlice[:, bold_columns])


In [5]:
unique_item_count = len(purchase_data["Item ID"].unique())
average_price = f"${round(purchase_data.Price.mean(), 2)}"
unique_purchase_count = len(purchase_data["Purchase ID"].unique())
total_revenue = f"${purchase_data.Price.sum()}"
purchase_data.head()
pd.DataFrame({
    "Number of Unique Items": unique_item_count,
    "Average Price": average_price,
    "Number of Purchases": unique_purchase_count,
    "Total Revenue": total_revenue
}, index=[0])

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,$2379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [6]:
player_gender_data = purchase_data[["SN", "Gender"]].drop_duplicates()
gender_demographics = player_gender_data.groupby("Gender").count()
gender_demographics = gender_demographics.rename(columns={
    "SN": "Total Count"
})
gender_demographics["Percentage of Players"] = gender_demographics["Total Count"] / unique_player_count * 100
gender_demographics = gender_demographics.loc[["Male", "Female", "Other / Non-Disclosed"], :]
gender_demographics.style.format("{:.2f}", pd.IndexSlice[:, "Percentage of Players"])

,Total Count,Percentage of Players
Gender,,
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [7]:
aggregate_gender_data = purchase_data.groupby("Gender")
aggregate_gender_player_data = purchase_data.groupby(["Gender", "SN"])

total_purchase_by_gender_person = aggregate_gender_player_data["Price"].sum()

gender_purchasing_analysis = pd.DataFrame({
    "Purchase Count": aggregate_gender_data["Purchase ID"].count(),
    "Average Purchase Price": aggregate_gender_data["Price"].mean(),
    "Total Purchase Value": aggregate_gender_data["Price"].sum(),
    "Avg Total Purchase per Person": total_purchase_by_gender_person.groupby(level=0).mean()
})

basicDFStyle(gender_purchasing_analysis, money_columns=["Average Purchase Price", "Total Purchase Value", "Avg Total Purchase per Person"])


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [8]:
bins = [0, *np.arange(9, 40, 5), 999]
labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]


purchase_data["Age Group"] = pd.cut(purchase_data["Age"], bins=bins, labels=labels)
unique_player_ages = purchase_data.groupby(["Age Group", "SN"])
age_counts = unique_player_ages["SN"].count().groupby(level=0).count()

age_demographics = pd.DataFrame({
    "Total Count": age_counts,
    "Percentage of Players": (age_counts / len(unique_players) * 100)
})
age_demographics.style.format("{:.2f}", pd.IndexSlice[:, "Percentage of Players"])

,Total Count,Percentage of Players
Age Group,,
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [9]:
aggregate_age_group = purchase_data.groupby("Age Group")

age_purchasing_analysis = pd.DataFrame({
    "Purchase Count": aggregate_age_group["Purchase ID"].count(),
    "Average Purchase Price": aggregate_age_group["Price"].mean(),
    "Total Purchase Value": aggregate_age_group["Price"].sum(),
    "Avg Total Purchase per Person": unique_player_ages["Price"].sum().groupby(level=0).mean(),
})
basicDFStyle(df=age_purchasing_analysis, money_columns=["Average Purchase Price", "Total Purchase Value", "Avg Total Purchase per Person"])


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [10]:
user_aggregate_df = purchase_data.groupby("SN")
top_spenders = pd.DataFrame({
    "Purchase Count": user_aggregate_df["Purchase ID"].count(),
    "Average Purchase Price": user_aggregate_df["Price"].mean(),
    "Total Purchase Value": user_aggregate_df["Price"].sum()
}).sort_values("Total Purchase Value", ascending=False)

basicDFStyle(df=top_spenders.head(), money_columns=["Average Purchase Price", "Total Purchase Value"])

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [11]:
item_aggregate = purchase_data.groupby("Item ID")

popular_items = pd.DataFrame({
    "Item Name": item_aggregate["Item Name"].first(),
    "Purchase Count": item_aggregate["Item Name"].count(),
    "Item Price": item_aggregate["Price"].first(),
    "Total Purchase Value":item_aggregate["Price"].sum()
}).sort_values("Purchase Count", ascending=False)

basicDFStyle(popular_items.head(), money_columns=["Item Price", "Total Purchase Value"], bold_columns=["Item Name"])

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [12]:
most_profitable_df = popular_items.copy()
most_profitable_df = most_profitable_df.sort_values("Total Purchase Value", ascending=False)
basicDFStyle(most_profitable_df.head(), ["Item Price", "Total Purchase Value"], ["Item Name"])

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
